# Intro to LLMstudio
In this tutorial you'll be running the proxy and the tracker so you can automatically log the usage metrics and costs in the default database (`llmstudio_mgmt.db`).

You'll learn:
1. How to connect to any provider available (VertexAI, OpenAI, etc.)
2. Make sync and async calls both with and without streaming
3. See the save logs

First things first:
* run `pip install llmstudio[proxy, tracker]`
* update your .env file with `GOOGLE_API_KEY` or `OPENAI_API_KEY`


## Start proxy and tracking servers
This step is to set the proxy and tracking servers.
You can opt out to start such servers (see `llmstudio-core` for the lightest version)

In [1]:
from pprint import pprint
from llmstudio.server import start_servers
from llmstudio_tracker.tracker import TrackingConfig
from llmstudio_proxy.provider import ProxyConfig

# start servers
start_servers(proxy=True, tracker=True)

proxy = ProxyConfig(host="0.0.0.0", port="8001")
tracking = TrackingConfig(host="0.0.0.0", port="8002")



Running LLMstudio Proxy on http://0.0.0.0:8001 
Running LLMstudio Tracking on http://0.0.0.0:8002 


In [2]:
from llmstudio.providers import LLM

# You can set OPENAI_API_KEY and GOOGLE_API_KEY on .env file or pass as api_key on LLM

vertexai = LLM("vertexai",
               proxy_config=proxy,
               tracking_config=tracking,
               session_id="vertexai-1")

openai = LLM("openai",
             proxy_config=proxy,
             tracking_config=tracking,
             session_id="openai-1")


Connected to LLMStudio Proxy @ 0.0.0.0:8001
Connected to LLMStudio Proxy @ 0.0.0.0:8001


# VertexAI
### Chat (non-stream)

In [3]:
response = vertexai.chat("What's your name", model="gemini-1.5-pro-latest")
print(response.chat_output)
pprint(response.metrics)

I don't have a name. I am a large language model, here to assist you with any questions or tasks you may have. 😊 

How can I help you today? 

{'cost_usd': 0.0004004,
 'input_tokens': 4,
 'inter_token_latency_s': 0.10699827330453056,
 'latency_s': 2.0291249752044678,
 'output_tokens': 38,
 'time_to_first_token_s': 1.248373031616211,
 'tokens_per_second': 3.9425861382411242,
 'total_tokens': 42}


### Chat Async (non-stream)

In [4]:
await vertexai.achat("What's your name", model="gemini-1.5-pro-latest")

ChatCompletion(id='c5d0d059-ace9-413a-b30c-8a339e0f9225', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I am a large language model, trained by Google. I don't have a name like humans do. \n\nHow can I assist you today? 😊 \n", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1729545819, model='gemini-1.5-pro-latest', object='chat.completion', service_tier=None, system_fingerprint=None, usage=None, chat_input="What's your name", chat_output="I am a large language model, trained by Google. I don't have a name like humans do. \n\nHow can I assist you today? 😊 \n", chat_output_stream='', context=[{'role': 'user', 'content': "What's your name"}], provider='vertexai', deployment='gemini-1.5-pro-latest', timestamp=1729545819.9707232, parameters={}, metrics={'input_tokens': 4, 'output_tokens': 32, 'total_tokens': 36, 'cost_usd': 0.00033739999999999996, 'latency_s': 1.7530958652496338, 'time_to_fir

### Chat (stream)

In [5]:
response = vertexai.chat("Write a paragfraph about space", model="gemini-1.5-pro-latest", is_stream=True)
for i, chunk in enumerate(response):
    if i%2==0:
        print("\n")
    if not chunk.metrics:
        print(chunk.chat_output_stream, end="", flush=True)
    else:
        print("\n\n## Metrics:")
        pprint(chunk.metrics)




Space, a boundless expanse of darkness sprinkled with the brilliance of celestial objects, has captivated

 humanity for millennia. From the twinkling stars in our Milky Way galaxy to the swirling gases of distant nebulas, the universe offers an endless panorama of wonder. It's a realm where gravity sculpts galaxies, where stars are born and

 extinguished in fiery explosions, and where planets, some potentially harboring life, orbit their suns in a delicate cosmic dance. Exploring this final frontier, pushing the boundaries of human knowledge and daring to venture into the unknown, continues to inspire us, fueling our curiosity about the origins of the universe and our place within it. 




## Metrics:
{'cost_usd': 0.0014518,
 'input_tokens': 8,
 'inter_token_latency_s': 0.22149918296120383,
 'latency_s': 3.5945589542388916,
 'output_tokens': 138,
 'time_to_first_token_s': 1.140388011932373,
 'tokens_per_second': 3.3383789646429287,
 'total_tokens': 146}


### Chat Async (stream)

In [6]:
i=0
async for chunk in await vertexai.achat("Write a paragfraph about space", model="gemini-1.5-pro-latest", is_stream=True):
    if i%20==2:
        print("\n")
    if not chunk.metrics:
        print(chunk.chat_output_stream, end="", flush=True)
    else:
        print("\n\n## Metrics:")
        pprint(chunk.metrics)
    i+=1


Space, a vast and enigmatic expanse, stretches out beyond the bounds of our planet,

 holding countless mysteries and wonders. Stars, like celestial diamonds, twinkle against the inky blackness, their light traveling for eons to reach our eyes. Planets, both familiar and alien, orbit distant suns, each with its own unique story to tell. Nebulas, clouds of gas and dust, paint the cosmos with vibrant hues, while galaxies, vast collections of stars, stretch out like celestial tapestries across the universe. From the smallest particles to the largest structures, space is a realm of endless fascination, inviting us to explore its depths and unravel its secrets. 


## Metrics:
{'cost_usd': 0.0014413,
 'input_tokens': 8,
 'inter_token_latency_s': 0.22311646288091486,
 'latency_s': 3.6461358070373535,
 'output_tokens': 137,
 'time_to_first_token_s': 1.1856458187103271,
 'tokens_per_second': 3.2911555232909797,
 'total_tokens': 145}


### Get session metrics and usage

In [7]:
from llmstudio_tracker.tracker import Tracker

tracker = Tracker(tracking_config=tracking)

In [15]:
logs = tracker.get_session_logs(session_id="vertexai-1")
logs.json()[0]


{'chat_input': "What's your name",
 'chat_output': "I don't have a name. I am a large language model, here to assist you with any questions or tasks you may have. 😊 \n\nHow can I help you today? \n",
 'session_id': 'vertexai-1',
 'context': [{'role': 'user', 'content': "What's your name"}],
 'provider': 'vertexai',
 'model': 'gemini-1.5-pro-latest',
 'deployment': 'gemini-1.5-pro-latest',
 'parameters': {},
 'metrics': {'input_tokens': 4,
  'output_tokens': 38,
  'total_tokens': 42,
  'cost_usd': 0.0004004,
  'latency_s': 2.0291249752044678,
  'time_to_first_token_s': 1.248373031616211,
  'inter_token_latency_s': 0.10699827330453056,
  'tokens_per_second': 3.9425861382411242},
 'log_id': 1,
 'created_at': '2024-10-21T21:23:38'}

# OpenAI
### Chat (non-stream)

In [9]:
response = openai.chat("What's your name", model="gpt-4o")
print(response.chat_output)
pprint(response.metrics)

I’m called Assistant, but you can name me whatever you’d like! How can I help you today?
{'cost_usd': 0.00035,
 'input_tokens': 4,
 'inter_token_latency_s': 0.014487525691156801,
 'latency_s': 1.730828046798706,
 'output_tokens': 22,
 'time_to_first_token_s': 1.3967618942260742,
 'tokens_per_second': 13.866195457364912,
 'total_tokens': 26}


### Chat Async (non-stream)

In [10]:
await openai.achat("What's your name", model="gpt-4o")

ChatCompletion(id='68984d4b-6a9e-4b62-9af5-195ab901ad83', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='I’m an AI language model created by OpenAI, and I don’t have a personal name. You can call me Assistant if you’d like!', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1729545829, model='gpt-4o', object='chat.completion', service_tier=None, system_fingerprint=None, usage=None, chat_input="What's your name", chat_output='I’m an AI language model created by OpenAI, and I don’t have a personal name. You can call me Assistant if you’d like!', chat_output_stream='', context=[{'role': 'user', 'content': "What's your name"}], provider='openai', deployment='gpt-4o-2024-08-06', timestamp=1729545830.260567, parameters={}, metrics={'input_tokens': 4, 'output_tokens': 30, 'total_tokens': 34, 'cost_usd': 0.00047, 'latency_s': 1.1592588424682617, 'time_to_first_token_s': 0.7933480739593506, 'inter_t

### Chat (stream)

In [11]:
response = openai.chat("Write a paragfraph about space", model="gpt-4o", is_stream=True)
for i, chunk in enumerate(response):
    if i%20==0:
        print("\n")
    if not chunk.metrics:
        print(chunk.chat_output_stream, end="", flush=True)
    else:
        print("\n\n## Metrics:")
        pprint(chunk.metrics)




Space, the vast expanse beyond our Earth's atmosphere, is an awe-inspiring realm that has

 captivated the human imagination for centuries. It is a place of endless wonder and profound mystery, where galaxies

, stars, planets, and celestial bodies float in a delicate cosmic dance. The exploration of space has

 led to groundbreaking discoveries, from the intricate details of our own solar system to the revelation of distant ex

oplanets that might harbor life. Space is not only a scientific frontier but also a source of philosophical

 contemplation, challenging us to ponder our place in the universe and inspiring a sense of unity among humankind

 as we look beyond our terrestrial home. As technology advances, space travel and research continue to push the

 boundaries of what we know, constantly redefining our understanding of the cosmos and our place within it.



## Metrics:
{'cost_usd': 0.0024850000000000002,
 'input_tokens': 8,
 'inter_token_latency_s': 0.011184079945087433,
 

### Chat Async (stream)

In [12]:
i=0
async for chunk in await openai.achat("Write a paragfraph about space", model="gpt-4o-mini", is_stream=True):
    if i%20==0:
        print("\n")
    if not chunk.metrics:
        print(chunk.chat_output_stream, end="", flush=True)
    else:
        print("\n\n## Metrics:")
        pprint(chunk.metrics)
    i+=1




Space, the vast and infinite expanse that exists beyond our planet, is a realm of wonder

 and mystery that captivates the human imagination. It is home to billions of galaxies, each containing countless

 stars and celestial bodies, including planets, moons, and asteroids. The sheer scale of space is

 difficult to comprehend, with distances measured in light-years, illuminating the vast chasms between stars.

 Within this cosmos, phenomena such as black holes, supernovae, and neutron stars challenge our understanding

 of physics and the universe's very nature. Beyond its physical attributes, space invites philosophical contemplation about our

 place in the universe, the possibility of extraterrestrial life, and the future of humanity as we continue

 to explore and expand our reach into the cosmos. As technology advances, our efforts to unveil the secrets

 of space may one day reveal answers to questions that have intrigued humanity for centuries.

## Metrics:
{'cost_usd': 0.000

In [13]:
from llmstudio_tracker.tracker import Tracker

tracker = Tracker(tracking_config=tracking)

In [14]:
logs = tracker.get_session_logs(session_id="openai-1")
logs.json()[0]

{'chat_input': "What's your name",
 'chat_output': 'I’m called Assistant, but you can name me whatever you’d like! How can I help you today?',
 'session_id': 'openai-1',
 'context': [{'role': 'user', 'content': "What's your name"}],
 'provider': 'openai',
 'model': 'gpt-4o',
 'deployment': 'gpt-4o-2024-08-06',
 'parameters': {},
 'metrics': {'input_tokens': 4,
  'output_tokens': 22,
  'total_tokens': 26,
  'cost_usd': 0.00035,
  'latency_s': 1.730828046798706,
  'time_to_first_token_s': 1.3967618942260742,
  'inter_token_latency_s': 0.014487525691156801,
  'tokens_per_second': 13.866195457364912},
 'log_id': 5,
 'created_at': '2024-10-21T21:23:49'}